In [2]:
import pandas as pd
import requests
import json

In [3]:
# 컬럼명 다 보이게 하기
pd.set_option('display.max_seq_items', None)

In [19]:
d = pd.read_csv('./CARD_SUBWAY_MONTH_202301.csv')
d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18797 entries, 20230101 to 20230131
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    18797 non-null  object 
 1   노선명     18797 non-null  object 
 2   역명      18797 non-null  int64  
 3   승차총승객수  18797 non-null  int64  
 4   하차총승객수  18797 non-null  int64  
 5   등록일자    0 non-null      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.0+ MB


In [20]:
d.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
20230101,경강선,곤지암,1450,1539,20230104,NaN
20230101,2호선,신도림,20210,21037,20230104,NaN
20230101,2호선,문래,5995,6567,20230104,NaN
20230101,2호선,영등포구청,5629,6117,20230104,NaN
20230101,2호선,당산,6821,7594,20230104,NaN


In [10]:
# 월별 지하철 승하차 정보 불러오고 합치기

combined_data = pd.DataFrame()

# 파일을 순회하며 데이터 합치기
for i in range(202301, 202312):
    file_name = f'CARD_SUBWAY_MONTH_{i}.csv'
    try:
        # CSV 파일 읽기
        data = pd.read_csv(file_name)
        
        # 읽어온 데이터를 기존 데이터에 추가
        combined_data = pd.concat([combined_data, data], ignore_index=True)
        print(f"File {file_name} added.")
    except FileNotFoundError:
        print(f"File {file_name} not found.")
    except Exception as e:
        print(f"An error occurred while processing {file_name}: {str(e)}")

# 결과를 하나의 CSV 파일로 저장
combined_data.to_csv('combined_subway_data_202301_to_202311.csv', index=False)
print("Combined data saved to combined_subway_data_202301_to_202311.csv")

File CARD_SUBWAY_MONTH_202301.csv added.
File CARD_SUBWAY_MONTH_202302.csv added.
File CARD_SUBWAY_MONTH_202303.csv added.
File CARD_SUBWAY_MONTH_202304.csv added.
File CARD_SUBWAY_MONTH_202305.csv added.
File CARD_SUBWAY_MONTH_202306.csv added.
File CARD_SUBWAY_MONTH_202307.csv added.
File CARD_SUBWAY_MONTH_202308.csv added.
File CARD_SUBWAY_MONTH_202309.csv added.
File CARD_SUBWAY_MONTH_202310.csv added.
File CARD_SUBWAY_MONTH_202311.csv added.
Combined data saved to combined_subway_data_202301_to_202311.csv


In [15]:
subway_data = pd.read_csv('combined_subway_data_202301_to_202311.csv')

In [16]:
subway_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203206 entries, 0 to 203205
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    203206 non-null  object 
 1   노선명     203206 non-null  object 
 2   역명      203206 non-null  int64  
 3   승차총승객수  203206 non-null  int64  
 4   하차총승객수  203206 non-null  int64  
 5   등록일자    0 non-null       float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.3+ MB


In [23]:
subway_data['사용일자'][:10]

0    경강선
1    2호선
2    2호선
3    2호선
4    2호선
5    2호선
6    2호선
7    2호선
8    2호선
9    2호선
Name: 사용일자, dtype: object

In [21]:
subway_data.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,경강선,곤지암,1450,1539,20230104,NaN
1,2호선,신도림,20210,21037,20230104,NaN
2,2호선,문래,5995,6567,20230104,NaN
3,2호선,영등포구청,5629,6117,20230104,NaN
4,2호선,당산,6821,7594,20230104,NaN


In [73]:
# 날씨 데이터 오픈 api 가져오기
url = 'https://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

In [92]:
payload = dict(serviceKey='CYfhWwsG3T6CbKfbZn%2BoWe5Ks2hKiIksiycfndf9OYy2l2XlQL4s1y1L5kDlVqe0JzyiTTRcsSO8Np03ikBXXA%3D%3D',
              pageNo='1',
               numOfRows='100',
               dataType = 'JSON',
               dateCd = 'HR',
               dataCd = 'ASOS',
               startDt = '20220101',
               startHh = '01',
               endDt = '20221130',
               endHh = '01',
               stnIds = '108'
              )

In [110]:
weather_list = []

for i in range(1,74):
    url = f'https://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=CYfhWwsG3T6CbKfbZn%2BoWe5Ks2hKiIksiycfndf9OYy2l2XlQL4s1y1L5kDlVqe0JzyiTTRcsSO8Np03ikBXXA%3D%3D&pageNo={i}&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20230101&startHh=01&endDt=20231101&endHh=01&stnIds=108'
    r = requests.get(url)

    for j in json.loads(r.text)['response']['body']['items']['item']:
        info = list(j.values())
        weather_list.append(info)

    

In [114]:
weather_info = pd.DataFrame(weather_list,columns=a)

In [115]:
# CSV 파일로 저장
weather_info.to_csv('data.csv', index=True)  # index=False를 지정하면 인덱스를 저장하지 않습니다

In [76]:
r = requests.get(url, params=payload)

In [77]:
r.text

'{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"tm":"2022-01-01 01:00","rnum":"1","stnId":"108","stnNm":"서울","ta":"-9.2","taQcflg":"","rn":"","rnQcflg":"","ws":"1.8","wsQcflg":"","wd":"270","wdQcflg":"","hm":"42","hmQcflg":"","pv":"1.3","td":"-19.7","pa":"1022.2","paQcflg":"","ps":"1033.6","psQcflg":"","ss":"","ssQcflg":"9","icsr":"","dsnw":"","hr3Fhsc":"","dc10Tca":"0","dc10LmcsCa":"0","clfmAbbrCd":"","lcsCh":"","vs":"2000","gndSttCd":"","dmstMtphNo":"","ts":"-7.2","tsQcflg":"","m005Te":"-1.1","m01Te":"-1.1","m02Te":"-0.2","m03Te":"1.0"},{"tm":"2022-01-01 02:00","rnum":"2","stnId":"108","stnNm":"서울","ta":"-9.5","taQcflg":"","rn":"","rnQcflg":"","ws":"1.2","wsQcflg":"","wd":"270","wdQcflg":"","hm":"43","hmQcflg":"","pv":"1.3","td":"-19.7","pa":"1022.3","paQcflg":"","ps":"1033.7","psQcflg":"","ss":"","ssQcflg":"9","icsr":"","dsnw":"","hr3Fhsc":"","dc10Tca":"0","dc10LmcsCa":"0","clfmAbbrCd":"","lcsCh":"","vs":"2000","gn

In [51]:
for i  in json.loads(r.text)["response"]['body']['items']['item']:
    print(list(i.values()))
    

['2022-01-01 01:00', '1', '108', '서울', '-9.2', '', '', '', '1.8', '', '270', '', '42', '', '1.3', '-19.7', '1022.2', '', '1033.6', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.2', '', '-1.1', '-1.1', '-0.2', '1.0']
['2022-01-01 02:00', '2', '108', '서울', '-9.5', '', '', '', '1.2', '', '270', '', '43', '', '1.3', '-19.7', '1022.3', '', '1033.7', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.5', '', '-1.3', '-1.2', '-0.2', '1.0']
['2022-01-01 03:00', '3', '108', '서울', '-9.3', '', '', '', '1.4', '', '290', '', '46', '', '1.4', '-18.8', '1022.6', '', '1034.0', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.6', '', '-1.4', '-1.2', '-0.2', '1.0']
['2022-01-01 04:00', '4', '108', '서울', '-9.6', '', '', '', '1.7', '', '340', '', '48', '', '1.4', '-18.5', '1021.9', '', '1033.3', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.6', '', '-1.4', '-1.3', '-0.3', '1.0']
['2022-01-01 05:00', '5', '108', '서울', '-9.7', '', '', '', '0.8', ''

In [78]:
weather_list = []
for i in json.loads(r.text)['response']['body']['items']['item']:
    info = list(i.values())
    weather_list.append(info)
    
print(weather_list)

[['2022-01-01 01:00', '1', '108', '서울', '-9.2', '', '', '', '1.8', '', '270', '', '42', '', '1.3', '-19.7', '1022.2', '', '1033.6', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.2', '', '-1.1', '-1.1', '-0.2', '1.0'], ['2022-01-01 02:00', '2', '108', '서울', '-9.5', '', '', '', '1.2', '', '270', '', '43', '', '1.3', '-19.7', '1022.3', '', '1033.7', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.5', '', '-1.3', '-1.2', '-0.2', '1.0'], ['2022-01-01 03:00', '3', '108', '서울', '-9.3', '', '', '', '1.4', '', '290', '', '46', '', '1.4', '-18.8', '1022.6', '', '1034.0', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.6', '', '-1.4', '-1.2', '-0.2', '1.0'], ['2022-01-01 04:00', '4', '108', '서울', '-9.6', '', '', '', '1.7', '', '340', '', '48', '', '1.4', '-18.5', '1021.9', '', '1033.3', '', '', '9', '', '', '', '0', '0', '', '', '2000', '', '', '-7.6', '', '-1.4', '-1.3', '-0.3', '1.0'], ['2022-01-01 05:00', '5', '108', '서울', '-9.7', '', '', '', '0.8

In [79]:
a = list(json.loads(r.text)['response']['body']['items']['item'][0].keys())

In [60]:
a

['tm',
 'rnum',
 'stnId',
 'stnNm',
 'ta',
 'taQcflg',
 'rn',
 'rnQcflg',
 'ws',
 'wsQcflg',
 'wd',
 'wdQcflg',
 'hm',
 'hmQcflg',
 'pv',
 'td',
 'pa',
 'paQcflg',
 'ps',
 'psQcflg',
 'ss',
 'ssQcflg',
 'icsr',
 'dsnw',
 'hr3Fhsc',
 'dc10Tca',
 'dc10LmcsCa',
 'clfmAbbrCd',
 'lcsCh',
 'vs',
 'gndSttCd',
 'dmstMtphNo',
 'ts',
 'tsQcflg',
 'm005Te',
 'm01Te',
 'm02Te',
 'm03Te']

In [80]:
pd.DataFrame(weather_list,columns=a)

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2022-01-01 01:00,1,108,서울,-9.2,,,,1.8,,...,,2000,,,-7.2,,-1.1,-1.1,-0.2,1.0
1,2022-01-01 02:00,2,108,서울,-9.5,,,,1.2,,...,,2000,,,-7.5,,-1.3,-1.2,-0.2,1.0
2,2022-01-01 03:00,3,108,서울,-9.3,,,,1.4,,...,,2000,,,-7.6,,-1.4,-1.2,-0.2,1.0
3,2022-01-01 04:00,4,108,서울,-9.6,,,,1.7,,...,,2000,,,-7.6,,-1.4,-1.3,-0.3,1.0
4,2022-01-01 05:00,5,108,서울,-9.7,,,,0.8,,...,,2000,,,-7.7,,-1.5,-1.4,-0.3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-01-05 00:00,96,108,서울,-5.6,,,9,1.3,,...,,2000,,,-5.7,,-0.5,-0.7,-0.3,0.7
96,2022-01-05 01:00,97,108,서울,-6.0,,,,1.3,,...,,2000,,,-6.1,,-0.6,-0.8,-0.3,0.7
97,2022-01-05 02:00,98,108,서울,-6.6,,,,2.2,,...,,2000,,,-6.2,,-0.7,-0.9,-0.3,0.7
98,2022-01-05 03:00,99,108,서울,-7.0,,,,1.0,,...,,2000,,,-6.7,,-0.8,-0.9,-0.3,0.7
